# Tensor Network Monte Carlo
Tensor Network Monte Carlo (TNMC) method generates biased samples from conditional probabilities computed via tensor network contractions and corrects the bias using the Metropolis scheme. Consequently, the proposals provided by tensor networks function as block updates for Monte Carlo simulations.

### Construct tensor network
First, we construct the tensor network for the simple Ising model.

![alt text](tensor-network.png)

In [134]:
using LinearAlgebra:svd, Diagonal, qr, norm
using TensorOperations
using OMEinsum

struct Parament
    L::Int
    chi::Int
    Beta::Float64
    Barray::Array{Float64, 2}
    I2::Array{Float64, 2}
    I3::Array{Float64, 3}
    I4::Array{Float64, 4}
    function Parament(L, chi, T)
        Beta = 1.0/T
        Barray = exp.(Beta * [1.0 -1.0; -1.0 1.0])
        I1 = vec([1.0 1.0])
        I2 = zeros(2, 2)
        I3 = zeros(2, 2, 2)
        I4 = zeros(2, 2, 2, 2)
        for i in 1:2
            I2[i ,i] = 1.0
            I3[i, i, i] = 1.0
            I4[i, i, i, i] = 1.0
        end
        new(L, chi, Beta, Barray, I2, I3, I4)
    end
end

mutable struct Spin
    Dx::Vector{Int}
    Dy::Vector{Int}
    bond::Array{Int, 3}

    function Spin(pa::Parament)
        Dx=[-1,0,1,0]
        Dy=[0,1,0,-1]
        bond = zeros(Int, pa.L, pa.L, 4)
        for y in 1:pa.L, x in 1:pa.L
            for k in 1:2
                jx,jy=x+Dx[k],y+Dy[k]
                if 0<jx<=pa.L && 0<jy<=pa.L 
                    bond[y,x,k] = 1
                    bond[jy,jx,k+2] = 1
                end
            end
        end
        new(Dx, Dy, bond)
    end
end


mutable struct Tensor
    Tensor::Array{Any,2}
end

function GetBond(sp::Spin, nnb::Int, y::Int, x::Int)
    return sp.bond[y, x, nnb] == 0 ? reshape([1], 1, 1) : pa.Barray
end

function Single(pa::Parament, sp::Spin, y::Int, x::Int, spin::Int)
    if y == 1 || y == pa.L
        if x == 1
            A = copy(reshape(pa.I2, 1, 2, 2))
        elseif x == pa.L
            A = copy(reshape(pa.I2, 2, 1, 2))  
        else  
            A = copy(pa.I3) 
        end
    else         
        if x == 1
            A = copy(reshape(pa.I3, 1, 2, 2, 2))  
        elseif x == pa.L
            A = copy(reshape(pa.I3, 2, 1, 2, 2))  
        else  
            A = copy(pa.I4)  
        end
    end
    indices = findall(x -> x == 1, A) 
    if spin==1  A[indices[1]] = 0 elseif spin==-1 A[indices[2]] = 0 end
    if y == pa.L
        A = ein"ijk,lj->ilk"(A, GetBond(sp, 3, y, x))
    elseif y == 1
        A = ein"ijk,lj,ka->ila"(A, GetBond(sp, 3, y, x),
        GetBond(sp, 2, y, x))         
    else
        A = ein"ijkl,nj,ka->inal"(A, GetBond(sp, 3, y, x),
        GetBond(sp, 2, y, x))  
    end   
    return A
end

function OutTensorGrid(pa::Parament, sp::Spin, te::Tensor)
    for y in 1:pa.L, x in 1:pa.L
        te.Tensor[y,x] = Single(pa, sp, y, x, 0)
    end
end         


OutTensorGrid (generic function with 1 method)

## Contraction the tensor network
Here we just contract the whole tensor network to get the partation function Z without store the process tensors.

![alt text](contraction.png)

In [135]:
function ContractlnZ(pa::Parament,te::Tensor)
    lnZ=0
    mps = Array{Any,1}(undef, pa.L)
    mps = deepcopy(te.Tensor[pa.L,:])
    for y in pa.L:-1:3
        res,mps = Compress(Eat(mps ,te.Tensor[y-1,:]),pa.chi)
        lnZ += res
    end
    res = Contruction(pa,te,te.Tensor[1,:],mps)
    lnZ += res
    return lnZ
end

function Contruction(pa::Parament, te::Tensor, MpsUp::Array{T}, MpsDw::Array{S}) where {T,S}
    lnZ = 0
    len = length(MpsUp)
    mps = Array{Any,1}(undef, len)
    mps[len] = ein"ikj,abj->iakb"(MpsUp[len],MpsDw[len])
    tnorm = norm(mps[pa.L])
    mps[pa.L] /= tnorm
    lnZ += log(tnorm)
    for i in pa.L:-1:2
        mps[i-1] =  ein"abij,nak,lbk->nlij"(mps[i],MpsUp[i-1],MpsDw[i-1])
        tnorm = norm(mps[i-1])
        mps[i-1] /= tnorm
        lnZ += log(tnorm)
    end
    return lnZ
end

function Eat(mps::Array{S},mpo::Array{O}) where {S,O}
    return [reshape(ein"ikj,abjc->aibkc"(mps[i],mpo[i]),size(mps[i],1)*size(mpo[i],1),:,2) for i in 1:length(mps)]
end

function Compress(mps::Array{T,S},chi::Int) where {T,S}
    residual=0
    len=length(mps)
    for i in 1:len 
        mps[i]=permutedims(mps[i],(1,3,2))
    end
    for i in 1:len-1
        F=qr(reshape(mps[i],size(mps[i],1)*2,:))
        Q=Matrix(F.Q)
        mps[i] = reshape(Q,size(mps[i],1),2,:)
        mps[i+1] = reshape(ein"ij,jab->iab"(F.R,mps[i+1]), size(F.R,1), 2, size(mps[i+1],3))  
        if mod(i,20) == 0
            tnorm = norm(mps[i+1])
            mps[i+1] /= tnorm
            residual += log(tnorm)
        end
    end   
    for i in len:-1:2
        F=svd(reshape(ein"ijk,kab->ijab"(mps[i-1],mps[i]), size(mps[i-1],1)*2,size(mps[i],3)*2))     
        if size(F.S,1)>chi
            Vt=F.Vt[1:chi,:]; U=F.U[:,1:chi]
            mps[i]=reshape(Vt, : , 2, size(mps[i],3))
            mps[i-1]=reshape(U*Diagonal(F.S[1:chi]), size(mps[i-1],1),2,:)
        else 
            mps[i]=reshape(F.Vt, :, 2, size(mps[i],3)) 
            mps[i-1]=reshape(F.U*Diagonal(F.S), size(mps[i-1],1),2,:)            
        end   
    end       
    tnorm=norm(mps[1])
    mps[1] /= tnorm
    residual += log(tnorm)
    for i in 1:len
        mps[i]=permutedims(mps[i],(1,3,2))
    end
    return residual,mps
end

Compress (generic function with 1 method)

## Sampling spins

$$P(s_i|\mathbf{s_{<i}})=\frac{\sum_{\mathbf{s_{>i}}}e^{-\beta E(s_i,\mathbf{s_{<i}})}}{\sum_{s_i,\mathbf{s_{>i}}}e^{-\beta E(s_i,\mathbf{s_{<i}})}}=\frac{Z(s_i,\mathbf{s_{<i}})}{\sum_{s_i}Z(s_i,\mathbf{s_{<i}})}.$$

In [136]:
function TNMH(pa::Parament, sp::Spin, te::Tensor)
    spin = zeros(Int, pa.L, pa.L)
    lnpr = 0; lnZq = zeros(2); lnZU = zeros(2)
    for y in 1:pa.L
        for x in 1:pa.L
            tensorup = Single(pa,sp,y,x,1)
            te.Tensor[y,x] .= tensorup
            lnZu = ContractlnZ(pa,te)

            tensordw = Single(pa,sp,y,x,-1)
            te.Tensor[y,x] .= tensordw
            lnZd = ContractlnZ(pa,te)
            pi= 1.0/(1.0+exp(lnZd-lnZu))

            if rand()<pi 
                spin[y,x]=1
                te.Tensor[y,x] .= tensorup
                lnpr += log(pi)
            else
                spin[y,x]=-1
                lnpr += log(1-pi)
            end
        end
    end
    return spin,lnpr
end

TNMH (generic function with 1 method)

## Markov Chain
Giving a current configuration $\mathbf{s}_a$, a candidate configuration $\mathbf{s}_b$ is generated from a proposal distribution $g(\mathbf{s}_b|\mathbf{s}_a)$, and is accepted with probability 
$$ p_a(\mathbf{s}_b|\mathbf{s}_a)=\min\left\{1,\frac{g(\mathbf{s}_a|\mathbf{s}_b)}{g(\mathbf{s}_b|\mathbf{s}_a)}\times\frac{P(\mathbf{s}_b)}{P(\mathbf{s}_a)}\right\} $$
which is called the Metropolis filter or the Metropolis acceptance-rejection scheme.  the acceptance rate of a configuration is exponential in its energy difference $$\frac{P(\mathbf{s}_b)}{P(\mathbf{s}_a)}=e^{\beta E(\mathbf{s}_a)-\beta E(\mathbf{s}_b)},$$
and the proposal distribution is independent $g(\mathbf{s}_b|\mathbf{s}_a)=g(\mathbf{s}_b)$

In [141]:
function GetEnergy(pa::Parament,sp::Spin,spin::Array{Int, 2})
    Energy = 0.0
    for y in 1:pa.L, x in 1:pa.L
        ns = spin[y, x]
        Nei = 0
        for k in 1:4
            jy = mod1(y + sp.Dy[k], pa.L)
            jx = mod1(x + sp.Dx[k], pa.L)
            Nei += spin[jy, jx] * sp.bond[y, x, k]
        end
        Energy += -ns * Nei/2
    end
    return Energy
end

function Accept(pa::Parament,sp::Spin,spinold::Array{Int,2},spinnew::Array{Int,2},lnprold::Float64,lnprnew::Float64)
    Energynew = GetEnergy(pa,sp,spinnew);  Energyold = GetEnergy(pa,sp,spinold)
    deltaE = Energynew-Energyold
    pi = exp(-pa.Beta*deltaE+lnprold-lnprnew)
    @show pi
    if pi>=1
        spinnext  = spinnew
        prnext= lnprnew
    else
        if rand()<pi
            spinnext = spinnew
            prnext = lnprnew
        else
            spinnext  = spinold
            prnext= lnprold
        end
    end
    return spinnext,prnext
end 

Accept (generic function with 1 method)

In [142]:
L = 4; chi = 2; T = 2.2

pa=Parament(L, chi, T)
sp=Spin(pa)
te=Tensor(Array{Any,2}(undef,pa.L,pa.L))

spinold = zeros(Int, pa.L, pa.L); spinnew = zeros(Int, pa.L, pa.L)
lnprold = 0.0; lnprnew = 0.0
OutTensorGrid(pa,sp,te)
spinold[:,:],lnprold=TNMH(pa,sp,te)   
for i in 1:10
    OutTensorGrid(pa,sp,te)
    spinnew[:,:],lnprnew=TNMH(pa,sp,te)                      
    spinold[:,:],lnprold=Accept(pa,sp,spinold[:,:],spinnew[:,:],lnprold,lnprnew)  
end



pi = 1.0046478268550647
pi = 0.9951898656791484
pi = 0.9993272400118214
pi = 1.0034089583633357
pi = 0.9965581726056333
pi = 0.999951134856162
pi = 0.9999999999999964
pi = 1.000255021716589
pi = 1.0000000000000018
pi = 0.9997450433029073
